For each driver/rider (person), put in the address and get the locations (long lat) through the geocode api (or any other api). We create a calculate_distance() function that takes in the long lat of 2 locations and returns the absolute distance for 2 locations.

For each driver, we calculate all riders within an arbitrary radius x. We greedily assign all passengers that can fit in this driver's car sorted by the greatest distance from the destination (church) to the furthest driver. We keep track of the distance traveled per passenger from their origin location to the order of their riders R_1, R_2, ... , R_m where m is the capacity of their car. If the amount of riders for their car is not filled, we reset the radius point to the last passenger picked up and 'shift' this origin point in the direction of the destination. We pick up riders on the way until the capacity m is filled.

We do this for a set of drivers. Each driver has a distance score for their certain car. We sum up the distance scores for all drivers after all riders have been assigned (if possible). 

We do this for all permutations of drivers. We use the assignment that minimizes the summed distance score for the drivers.